# A brief example on how to extract a single time step from a zarr store

Includes a clarification when loading to memory is actually triggered.

In [1]:
import numpy as np
import xarray as xr
import zarr

In [2]:
import dask
dask.config.set({"array.slicing.split_large_chunks": True})

from dask.distributed import Client
client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:42715 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 67.11 GB


In [4]:
# load a zarr store; function taken from calc_ccf_sampling.ipynb
def load_var_ddt_temp_rad_fromflux(expid, var):
    # open previously calculated radiative heating rates from zarr store
    zarr_store = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'+expid+'_ddttemp_rad-from-fluxes_DOM01_ML.zarr'
    return ( xr.open_zarr(zarr_store)
             [var].resample(time="1H").nearest(tolerance="5M").squeeze() )

At this point, ds is not loaded to disk. This is dask's lazy data access model, and can be seen by the fact that there are 442 tasks associated with the dataset.

In [5]:
ds = load_var_ddt_temp_rad_fromflux('nawdexnwp-2km-mis-0001', 'ddt_temp_radlw_fromflux')

In [6]:
ds

,Array,Chunk
Bytes,237.15 GB,600.00 MB
Shape,"(97, 75, 8149456)","(2, 75, 1000000)"
Count,442 Tasks,441 Chunks
Type,float32,numpy.ndarray


We can request loading the data to memory via compute, this will trigger the computation of the tasks. But it will lead to a memory error on the compute node use here as the dataset has a size of ~240GB.

In [7]:
ds.compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('zarr-e8c768b4bc8ca98128410b3f5b91bd26', 16, 0, 6)", <Worker 'tcp://127.0.0.1:40298', name: 3, memory: 0, processing: 75>)

We can circumvent this by selecting one time step before requiring a compute. A single timestep has a size of 2.4 GB, so no problem at all.

In [8]:
ds.isel(time=0)

<xarray.DataArray 'ddt_temp_radlw_fromflux' (height: 75, ncells: 8149456)>
dask.array<getitem, shape=(75, 8149456), dtype=float32, chunksize=(75, 1000000), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 2016-09-22
  * height   (height) float64 1.0 2.0 3.0 4.0 5.0 ... 71.0 72.0 73.0 74.0 75.0
Dimensions without coordinates: ncells

And now the compute runs fine.

In [9]:
ds.isel(time=0).compute()

<xarray.DataArray 'ddt_temp_radlw_fromflux' (height: 75, ncells: 8149456)>
array([[-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -1.6445528e-04, -1.6440985e-04, -1.6446011e-04],
       [-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -7.6336895e-05, -7.6291828e-05, -7.6369710e-05],
       [-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -5.1205578e-05, -5.1154202e-05, -5.1236624e-05],
       ...,
       [-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -1.3090692e-04, -1.4218441e-04, -1.3086860e-04],
       [-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -2.3419174e-04, -2.3412003e-04, -2.3408930e-04],
       [-0.0000000e+00, -0.0000000e+00, -0.0000000e+00, ...,
        -2.3601404e-04, -2.3657153e-04, -2.3555857e-04]], dtype=float32)
Coordinates:
    time     datetime64[ns] 2016-09-22
  * height   (height) float64 1.0 2.0 3.0 4.0 5.0 ... 71.0 72.0 73.0 74.0 75.0
Dimensions without coordinates: ncells